In [1]:
from flask import Flask, make_response, request, render_template
import io
from io import StringIO
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import os
from keras.models import load_model
from flask_preprocess import preprocess

app = Flask(__name__)

In [2]:
@app.route('/')
def form():
    return """
        <html>
            <body>
                <h1>COVID19 Severity Prediction</h1>
                </br>
                </br>
                <p> Insert your CSV file and then download the result
                <form action="/predict" method="post" enctype="multipart/form-data">
                    <input type="file" name="data_file" class="btn btn-block"/>
                    </br>
                    </br>
                    <button type="submit" class="btn btn-primary btn-block btn-large">Predict</button>
                </form>
            </body>
        </html>
    """

In [3]:
@app.route('/predict', methods=["POST"])
def transform_view():
    if request.method == 'POST':
        f = request.files['data_file']
        if not f:
            return "No file"
    
    stream = io.StringIO(f.stream.read().decode("UTF8"), newline=None)
    csv_input = csv.reader(stream)
    stream.seek(0)
    result = stream.read()
    data = pd.read_csv(StringIO(result))
    model = load_model('model_percobaan', compile=False)
    df = preprocess(data)
    prediction = model.predict(df)
    df_predict = pd.DataFrame(prediction, columns=[['Severity_Mild','Severity_Moderate','Severity_None','Severity_Severe']])
    df_predict.to_csv("prediction.csv", index=False, header=False, encoding='utf8')
    
    response = make_response(df_predict.to_csv())
    response.headers["Content-Disposition"] = "attachment; filename=result.csv"
    return response

In [4]:
if (__name__ == "__main__"):
     app.run(port = 5000, use_reloader=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [03/Jun/2022 22:51:36] "GET / HTTP/1.1" 200 -


1980/1980 [==============================] - 3s 1ms/step


127.0.0.1 - - [03/Jun/2022 22:51:47] "POST /predict HTTP/1.1" 200 -
